# Prepare a Protein Ligand system for vDSSB and standard NEW-FSDAM

This is a template, preparing a system for openmm can be complex and there can be a lot of variables to consider, therefore you might need to adapt it to your needs

And remember to read everything carefully, this process is far from being automatic!!

In [ ]:
import pdbfixer
from simtk import unit, openmm
from simtk.openmm import app, Platform, LangevinIntegrator, XmlSerializer
from simtk.openmm.app import PDBFile, Simulation, Modeller, StateDataReporter, ForceField
import mdtraj
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import GAFFTemplateGenerator, SMIRNOFFTemplateGenerator
import numpy as np
from openmmtools import testsystems
from openmmtools.integrators import VelocityVerletIntegrator
from openmmtools.states import ThermodynamicState
import parmed
import sys
import time

In [ ]:
#this 2 files must be given as input
protein_ligand_pdb = ''
#atoms must be in the same order as in the protein_ligand_pdb (you can help yourself with openbabel)
ligand_sdf_file = ''
ligand_resname = ''

#this files will be created by the script
only_protein_pdb = 'only_protein.pdb'
only_ligand_pdb = 'only_ligand.pdb'

In [ ]:
protein_forcefields = ['amber/ff14SB.xml']
water_forcefields = ['amber/tip3p_standard.xml']
water_type = 'tip3p'
ligand_forcefield = 'gaff-2.11'

## Repair the PDB file if needed
also changes atom and residue names to standard ones

In [ ]:
def pdb_repair(input_file_name, output_file_name, add_H=True, ph = 7.0):
    """repairs a PDB file with pdbfixer

    input_file_name : str
        the pdb or mmcif file to be repaired
    output_file_name : str
        the name of the new structure file that will be created
    add_H : bool, default True
        if True pdbfixer will add hydrogens according to ph
    ph : float, default 7.0
        if add_H == True this is the pH value that will be used to add hydrogens
    """

    with open(input_file_name, 'r') as f:
        fixer = pdbfixer.PDBFixer(pdbfile = f)

    fixer.findMissingResidues()

    fixer.findNonstandardResidues()

    fixer.replaceNonstandardResidues()

    #fixer.removeHeterogens(False)

    fixer.findMissingAtoms()

    fixer.addMissingAtoms()

    if add_H:
        fixer.addMissingHydrogens(ph)

    #fixer.addSolvent(fixer.topology.getUnitCellDimensions())

    
    with open(output_file_name, 'w') as f:
        PDBFile.writeFile(fixer.topology, fixer.positions, f, keepIds = True)


In [ ]:
#will overwrite the input file!!!
pdb_repair(protein_ligand_pdb, protein_ligand_pdb, add_H=True, ph = 7.0)

# divide in protein and ligand pdb

Change the selection strings accordingly to your system, 'protein' often doesn't work as expected!

In [ ]:
trj = mdtraj.load(protein_ligand_pdb)

#ATTENTION! the 'protein' selection works only in simple cases with no ions involved
# adapt the selection string to your needs!
trj.atom_slice(trj.top.select('protein')).save(only_protein_pdb)
trj.atom_slice(trj.top.select(f'resname {ligand_resname}')).save(only_ligand_pdb)

del trj

# The function to equilibrate

Equilibrating on a Jupyter notebook can be VERY slow, do it only if you think it is a good idea

In [ ]:
#This function has been readapted from
# https://github.com/tdudgeon/simple-simulate-complex
def equilibrate(input_pdb,
                system_xml,
                output_pdb='output.pdb',
                timestep=1.5 * unit.femtoseconds,
                nsteps=3500,
                minimize=True,
                temperature=298.15*unit.kelvin,
                pressure=1.0*unit.atmosphere):
    
    pdb = PDBFile(input_pdb)

    with open(system_xml, 'r') as fp:

        string = ''.join(fp.readlines())
        system = openmm.XmlSerializer.deserialize(string)
    
    print('Use PBC:', system.usesPeriodicBoundaryConditions())

    integrator = VelocityVerletIntegrator(timestep)

    term_state = ThermodynamicState(system=system,
                                temperature=temperature,
                                pressure=pressure)

    print('Default Periodic box vectors:', system.getDefaultPeriodicBoxVectors())

    simulation = Simulation(pdb.topology, term_state.system, integrator)
    context = simulation.context
    context.setPositions(pdb.positions)
    term_state.apply_to_context(context)

    if minimize:
        print('Minimising ...')
        simulation.minimizeEnergy()


    # equilibrate
    simulation.context.setVelocitiesToTemperature(term_state.temperature)

    term_state.apply_to_context(context)
    print('Equilibrating ...')

    # Run the simulation.
    # The enforcePeriodicBox arg to the reporters is important.
    # It's a bit counter-intuitive that the value needs to be False, but this is needed to ensure that
    # all parts of the simulation end up in the same periodic box when being output.
    # simulation.reporters.append(PDBReporter(output_traj_pdb, reporting_interval, enforcePeriodicBox=False))
    #simulation.reporters.append(DCDReporter('output_traj.dcd', 1000, enforcePeriodicBox=False))
    simulation.reporters.append(StateDataReporter(sys.stdout, 100, step=True, potentialEnergy=True, temperature=True))
    t0 = time.time()
    simulation.step(nsteps)
    t1 = time.time()
    print('Simulation complete in', t1 - t0, 'seconds at')


    with open(output_pdb, 'w') as outfile:
        PDBFile.writeFile(pdb.topology, context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(), file=outfile, keepIds=True)

# a useful function

OpenMM loves to have the atom coordinates starting from 0,0,0 and therefore this function will be used as no tomorrow

In [ ]:
def scale_positions_and_update_box_vectors(positions, topology):

    coordinates = np.array(positions / unit.nanometers)

    max_coord = []
    for i in range(3):

        coordinates[:, i] = coordinates[:, i] - np.amin(coordinates[:, i])

        max_coord.append(np.amax(coordinates[:, i] + 0.00001))

    coordinates = coordinates *  unit.nanometers

    positions = coordinates

    topology.setUnitCellDimensions(max_coord * unit.nanometers)

    return positions, topology

# The ligand

## you must choose what you want

If you want a ligand solvated in water in order to do a standard NEW-FSDAM or a ligand in vacuum with another pdb of a box of water and a gromacs topology of the ligand solvated in this water in order to do vDSSB you will have to run different cells

In [ ]:
mol = Molecule.from_file(ligand_sdf_file)

mol_positions, mol_topology =  scale_positions_and_update_box_vectors(mol.conformers[0], mol.to_topology().to_openmm())

In [ ]:
print(GAFFTemplateGenerator.INSTALLED_FORCEFIELDS)
print('\n')
print(SMIRNOFFTemplateGenerator.INSTALLED_FORCEFIELDS)

In [ ]:
if 'gaff' in ligand_forcefield:
    ligand_ff_generator = GAFFTemplateGenerator(mol, forcefield=ligand_forcefield)
else:

    ligand_ff_generator = SMIRNOFFTemplateGenerator(mol, forcefield=ligand_forcefield)

In [ ]:
system_kwargs = {'constraints': app.HBonds, 'rigidWater': True, 'nonbondedMethod': app.PME}#, 'nonbondedCutoff':0.4*unit.nanometer}

# if you want to do vDSSB run this
(ligand gets created in water)

In [ ]:
mol_topology.setUnitCellDimensions([50.0,50.0,50.0] * unit.nanometers)

ligand_ff = ForceField()

ligand_ff.registerTemplateGenerator(ligand_ff_generator.generator)

ligand_system = ligand_ff.createSystem(mol_topology, **system_kwargs)

with open('system_only_ligand.xml', 'w') as f:
    f.writelines(XmlSerializer.serialize(ligand_system))

In [ ]:
water_box = testsystems.WaterBox(box_edge=3.0*unit.nanometers, model=water_type,
                                 constrained=False,nonbondedMethod=app.PME, ionic_strength=0*unit.molar)

with open('only_water.pdb', 'w') as outfile:
    PDBFile.writeFile(water_box.topology, water_box.positions, file=outfile)

with open('system_only_water.xml', 'w') as f:
    f.writelines(XmlSerializer.serialize(water_box.system))

In [ ]:
ligand_modeller = Modeller(water_box.topology, water_box.positions)

ligand_modeller.add(mol_topology, mol_positions)

ligand_ff = ForceField(*water_forcefields)
ligand_ff.registerTemplateGenerator(ligand_ff_generator.generator)

ligand_unrestrained_system = ligand_ff.createSystem(ligand_modeller.topology, **{'constraints': None, 'rigidWater': False, 'nonbondedMethod': app.PME})

pmd_structure = parmed.openmm.load_topology(ligand_modeller.topology,
system=ligand_unrestrained_system, xyz=ligand_modeller.positions)

pmd_structure.save('solv_ligand_top_for_alchemical_creation.top', overwrite=True)

del ligand_modeller
del pmd_structure
del ligand_unrestrained_system
del ligand_ff
del ligand_system

In [ ]:
equilibrate(input_pdb=only_ligand_pdb,
                system_xml='system_only_ligand.xml',
                output_pdb='only_ligand_equilibrated.pdb',
           pressure=None)

In [ ]:
equilibrate(input_pdb='only_water.pdb',
                system_xml='system_only_water.xml',
                output_pdb='only_water.pdb')

# If you want to do standard NEW-FSDAM run this
(ligand gets annihilated from a box of water)

In [ ]:
ligand_ff = ForceField(*water_forcefields)

ligand_ff.registerTemplateGenerator(ligand_ff_generator.generator)

ligand_modeller = Modeller(mol_topology, mol_positions)

ligand_modeller.addSolvent(ligand_ff, model=water_type, padding=1.0*unit.nanometers, neutralize=False)

ligand_modeller.positions, ligand_modeller.topology =  scale_positions_and_update_box_vectors(ligand_modeller.positions, ligand_modeller.topology)

ligand_system = ligand_ff.createSystem(ligand_modeller.topology, **system_kwargs)

with open('system_solvated_ligand.xml', 'w') as f:
    f.writelines(XmlSerializer.serialize(ligand_system))
    
with open('solvated_ligand.pdb', 'w') as outfile:
    PDBFile.writeFile(ligand_modeller.topology, ligand_modeller.positions, file=outfile)
    
del ligand_modeller
del ligand_ff
del ligand_system

In [ ]:
equilibrate(input_pdb='solvated_ligand.pdb',
                system_xml='system_solvated_ligand.xml',
                output_pdb='solvated_ligand.pdb')

# And now the protein ligand complex
(In this part it is important that the ligand sdf file and the input pdb file have the atoms in the same order but the xyz positions will be taken from the pdb so don't worry if the sdf has random positions)

In [ ]:
protein_ligand_ff = ForceField(*list(water_forcefields + protein_forcefields))

protein_ligand_ff.registerTemplateGenerator(ligand_ff_generator.generator)

protein_pdb = PDBFile(only_protein_pdb)
ligand_pdb = PDBFile(only_ligand_pdb)

protein_ligand_modeller = Modeller(protein_pdb.topology, protein_pdb.positions)
protein_ligand_modeller.add(mol_topology, ligand_pdb.positions)

protein_ligand_modeller.positions, protein_ligand_modeller.topology =  scale_positions_and_update_box_vectors(protein_ligand_modeller.positions, protein_ligand_modeller.topology)

protein_ligand_modeller.addSolvent(protein_ligand_ff, model=water_type, padding=1.0*unit.nanometers, neutralize=False)

protein_ligand_modeller.positions, protein_ligand_modeller.topology =  scale_positions_and_update_box_vectors(protein_ligand_modeller.positions, protein_ligand_modeller.topology)

protein_ligand_system = protein_ligand_ff.createSystem(protein_ligand_modeller.topology, **system_kwargs)

with open('system_protein_ligand.xml', 'w') as f:
    f.writelines(XmlSerializer.serialize(protein_ligand_system))
    
with open('new_equilibrated_protein_ligand.pdb', 'w') as outfile:
    PDBFile.writeFile(protein_ligand_modeller.topology, protein_ligand_modeller.positions, file=outfile)

In [ ]:
equilibrate(input_pdb='new_equilibrated_protein_ligand.pdb',
                system_xml='system_protein_ligand.xml',
                output_pdb='new_equilibrated_protein_ligand.pdb')

# Done, you are ready to go